<h1 style="color:#1f77b4; text-align:left; font-size:40px;">
    Data Cleaning
</h1>

<h3 style="color:#555; text-align:left;">
    <strong>Purpose:</strong><br><br>
    - Load the unified master dataset *telco_master* created in notebook 002.<br><br>
    - Perform <strong>light, transparent cleaning</strong> <em>before</em> EDA:
    <ul>
        <li>Remove clearly redundant or uninformative columns</li>
        <li>Standardize key types and strip whitespace</li>
        <li>Run basic structure and missing-value checks</li>
    </ul>
    - Use <strong>simple visualizations and summaries</strong> to justify why some columns are dropped.<br><br>
    - Save a “basic cleaned” dataset for EDA: *telco_master_basic_clean*
</h3>

<h2 style="color:#1f77b4; border-bottom: 3px solid #1f77b4; padding-bottom:4px;">
</h2>


# Imports

In [1]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
src_path = project_root / "src"
sys.path.append(str(src_path))

from utils_data import load_df, quick_overview

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Load Interim Data

In [2]:
MASTER_NAME = "telco_master"

df = load_df(MASTER_NAME, folder="interim")
print(f"\nLoaded '{MASTER_NAME}' from interim with shape: {df.shape}")


📂 Loaded: /Users/dianagomes/Desktop/work/s2/EnterpriseDataScienceBootcamp_workgroup/data/interim/telco_master.csv

Loaded 'telco_master' from interim with shape: (7043, 56)


# Quick Overview

In [3]:
quick_overview(df, name="telco_master", show_head=True, n_head=5)


===== telco_master =====
Shape: 7043 rows × 56 columns

Data types:
demo_count                                 int64
demo_gender                               object
demo_age                                   int64
demo_under_30                             object
demo_senior_citizen                       object
demo_married                              object
demo_dependents                           object
demo_number_of_dependents                  int64
loc_count                                  int64
loc_country                               object
loc_state                                 object
loc_city                                  object
loc_zip_code                               int64
loc_lat_long                              object
loc_latitude                             float64
loc_longitude                            float64
svc_count                                  int64
svc_quarter                               object
svc_referred_a_friend                     object


,demo_count,demo_gender,demo_age,demo_under_30,demo_senior_citizen,demo_married,demo_dependents,demo_number_of_dependents,loc_count,loc_country,...,st_quarter,st_satisfaction_score,st_customer_status,st_churn_label,st_churn_value,st_churn_score,st_cltv,st_churn_category,st_churn_reason,zip_population
0,1,Male,78,No,Yes,No,No,0,1,United States,...,Q3,3,Churned,Yes,1,91,5433,Competitor,Competitor offered more data,68701
1,1,Female,74,No,Yes,Yes,Yes,1,1,United States,...,Q3,3,Churned,Yes,1,69,5302,Competitor,Competitor made better offer,55668
2,1,Male,71,No,Yes,No,Yes,3,1,United States,...,Q3,2,Churned,Yes,1,81,3179,Competitor,Competitor made better offer,47534
3,1,Female,78,No,Yes,Yes,Yes,1,1,United States,...,Q3,2,Churned,Yes,1,88,5337,Dissatisfaction,Limited range of services,27778
4,1,Female,80,No,Yes,Yes,Yes,1,1,United States,...,Q3,2,Churned,Yes,1,67,2793,Price,Extra data charges,26265


# Phase 1 – Structural & Integrity Checks

 Verify the integrity of the merged dataset before any transformations:

## Inspect variables

In [4]:
dtype_summary = (
    df.dtypes
    .reset_index()
    .rename(columns={'index': 'column', 0: 'dtype'})
    .sort_values('dtype')
)
display(dtype_summary)

,column,dtype
0,demo_count,int64
52,st_cltv,int64
51,st_churn_score,int64
50,st_churn_value,int64
47,st_satisfaction_score,int64
45,st_count,int64
42,svc_total_extra_data_charges,int64
20,svc_tenure_in_months,int64
19,svc_number_of_referrals,int64
16,svc_count,int64


### Grouping the variables for analysis

In [5]:
demographic_vars = [c for c in df.columns if c.startswith("demo_")]
location_vars    = [c for c in df.columns if c.startswith("loc_")]
service_vars     = [c for c in df.columns if c.startswith("svc_")]
status_vars      = [c for c in df.columns if c.startswith("st_")]

In [6]:
for group, cols in {
    "Demographics": demographic_vars,
    "Location": location_vars,
    "Services": service_vars,
    "Status": status_vars
}.items():
    print(f"\n{group} ({len(cols)} vars)")
    print(cols)


Demographics (8 vars)
['demo_count', 'demo_gender', 'demo_age', 'demo_under_30', 'demo_senior_citizen', 'demo_married', 'demo_dependents', 'demo_number_of_dependents']

Location (8 vars)
['loc_count', 'loc_country', 'loc_state', 'loc_city', 'loc_zip_code', 'loc_lat_long', 'loc_latitude', 'loc_longitude']

Services (29 vars)
['svc_count', 'svc_quarter', 'svc_referred_a_friend', 'svc_number_of_referrals', 'svc_tenure_in_months', 'svc_offer', 'svc_phone_service', 'svc_avg_monthly_long_distance_charges', 'svc_multiple_lines', 'svc_internet_service', 'svc_internet_type', 'svc_avg_monthly_gb_download', 'svc_online_security', 'svc_online_backup', 'svc_device_protection_plan', 'svc_premium_tech_support', 'svc_streaming_tv', 'svc_streaming_movies', 'svc_streaming_music', 'svc_unlimited_data', 'svc_contract', 'svc_paperless_billing', 'svc_payment_method', 'svc_monthly_charge', 'svc_total_charges', 'svc_total_refunds', 'svc_total_extra_data_charges', 'svc_total_long_distance_charges', 'svc_total

---
**Services variables**

Each service variable was reviewed for business interpretability, temporal validity, and expected influence on churn. The variables capture 5 behavioral dimensions:
- **Customer relationship & engagement**
  - svc_tenure_in_months, svc_offer, 
  - svc_referred_a_friend, svc_number_of_referrals, svc_contract

- **Service subscriptions**
  - svc_phone_service, svc_multiple_lines
  - svc_internet_service, svc_internet_type
  - svc_streaming_tv, svc_streaming_movies, svc_streaming_music
  - svc_unlimited_data
  - svc_online_security, svc_online_backup
  - svc_device_protection_plan, svc_premium_tech_support

- **Usage intensity**
  - svc_avg_monthly_long_distance_charges
  - svc_avg_monthly_gb_download

- **Billing and payment behavior**
  - svc_paperless_billing, svc_payment_method, svc_monthly_charge

- ❌ **Revenue and financial exposure**
  - High leakage risk: likely to encode information **after** the churn event.
       
    Example:

    (Low total revenue → customer must have churned early)
       
    (High total revenue → customer stayed longer)

  - svc_total_charges, svc_total_refunds,
    svc_total_extra_data_charges, svc_total_long_distance_charges,
    svc_total_revenue

- **Time / metadata**
  - svc_quarter, svc_count

In [22]:
leakage_vars = [
    'svc_total_revenue',
    'svc_total_charges',
    'svc_total_refunds',
    'svc_total_extra_data_charges',
    'svc_total_long_distance_charges'
]

# Keep only those that actually exist
leakage_vars = [c for c in leakage_vars if c in service_vars.columns]
leakage_vars

AttributeError: 'list' object has no attribute 'columns'

In [20]:
corr_vars = ['svc_tenure_in_months'] + leakage_vars
corr_vars = [c for c in corr_vars if c in df_svc.columns]

plt.figure(figsize=(6, 4))
sns.heatmap(service_vars[corr_vars].corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Between Tenure and Cumulative Financial Variables")
plt.tight_layout()
plt.show()

NameError: name 'leakage_vars' is not defined

---
**Target & Churn Outcome Variables**

Using these variables in training will cause data leakage and unrealistically high model performance.
- ✅ **st_churn_label** *(object)*  **will be used for validation**
  → Target (Yes/No churn) 

- ❌ **st_count** *(int64)*  
  → No variation (1)

- ❌ **st_quarter** *(object)*  
  → Time reference tied to churn outcome (Q3)

- ⚠️ **st_satisfaction_score** *(object)*  
  → Customer satisfaction rating (1–5)  
  Potential risk: proceed with caution

- ❌ **st_customer_status** *(object)*  
  → Current state (Active/Churned)

- ❌ **st_churn_value** *(int64)*  
  → Encoded churn status (binary)

- ❌ **st_churn_score** *(int64)*  
  → Likely a precomputed churn risk score

- ❌ **st_cltv** *(object)*  
  → Custumer lifetime value

- ❌ **st_churn_category** *(object)*  
  → Why they churned (Competitor, Price, Service, etc.)

- ❌ **st_churn_reason** *(object)*  
  → Detailed textual reason


### Numeric Columns (19 total)

---

#### `demo_count`
- Likely household or record count  
- ⚠️ Meaning requires clarification  
- 🔧 *Action:* Retain only if definition is confirmed

---

#### `st_cltv`
- Customer lifetime value  
- ⚠️ **Potential leakage risk** (depends on when it is calculated)  
- 🔧 *Action:* Exclude unless confirmed to be based only on pre-churn history

---

#### `st_churn_score`
- Internal churn risk score (0–100)  
- ⚠️ **High-risk warning:** Potential **target leakage**  
- 🔧 *Action:* Must be excluded from predictive modeling unless explicitly justified as pre-label information

---

#### `st_churn_value`
- Monetary value associated with churn  
- ❌ **Direct target leakage**  
- 🔧 *Action:* Must be removed from the feature set

---

#### `st_satisfaction_score`
- Customer satisfaction rating (1–5)  
- ✅ Valid numeric feature  
- 🔧 *Action:* Check correlation with churn

---

#### `st_count`
- Unclear status-related count feature  
- ⚠️ Ambiguous meaning  
- 🔧 *Action:* Verify definition before use

---

#### `svc_total_extra_data_charges`
- Charges due to data overages  
- ✅ Valid numeric feature  
- 🔧 *Action:* Check prevalence of zeros (possible class imbalance)

---

#### `svc_tenure_in_months`
- Customer tenure  
- ✅ Valid numeric feature  
- 🔧 *Action:* Likely strong churn driver — consider binning for interpretability

---

#### `svc_number_of_referrals`
- Number of friends referred  
- ✅ Valid numeric feature  
- 🔧 *Action:* Likely right-skewed — consider log transform or binning

---

#### `svc_count`
- Number of services subscribed by the customer  
- ✅ Important churn predictor  
- 🔧 *Action:* Keep as-is

---

#### `loc_zip_code`
- ZIP code identifier  
- ⚠️ **Not a true numeric feature**  
- 🔧 *Action:* Convert to string to preserve leading zeros and avoid misleading numerical ordering


In [ ]:
df["loc_zip_code"] = df["loc_zip_code"].astype(str)

---

#### `svc_avg_monthly_gb_download`
- Internet data usage  
- ✅ Valid numeric feature  
- 🔧 *Action:* Inspect distribution for outliers

---

#### `zip_population`
- Population by ZIP code  
- ✅ Valid numeric feature  
- 🔧 *Action:* Optional scaling during modeling

---

#### `demo_age`
- Customer age  
- ✅ Strong churn-related demographic feature  
- 🔧 *Action:* Consider binning for interpretability (e.g., <30, 30–50, 50+)

---

#### `loc_count`
- Number of records or entities per location  
- ⚠️ Requires clarification  
- 🔧 *Action:* Retain only if definition is confirmed

---

#### `demo_number_of_dependents`
- Number of dependents  
- ✅ Valid numeric feature  
- 🔧 *Action:* None required

---

#### `svc_monthly_charge`
- Monthly service fee  
- ✅ Strong financial churn driver  
- 🔧 *Action:* Apply scaling for distance-based models

---

#### `svc_total_charges`
- Total amount billed to the customer  
- ✅ Valid cumulative financial feature  
- 🔧 *Action:* Check correlation with revenue and monthly charges

---

#### `svc_total_refunds`
- Total amount refunded  
- ✅ Valid numeric feature  
- 🔧 *Action:* Inspect for sparsity and outliers

---

#### `svc_total_long_distance_charges`
- Total long-distance call charges  
- ✅ Valid numeric feature  
- 🔧 *Action:* Scale and inspect skewness

---

#### `svc_total_revenue`
- Total net revenue generated  
- ✅ Valid feature but highly correlated with total charges  
- 🔧 *Action:* Consider removing during multicollinearity checks

---

#### `svc_avg_monthly_long_distance_charges`
- Average long-distance charge per month  
- ✅ Valid numeric feature  
- 🔧 *Action:* Scale before modeling

---

#### `loc_longitude`
- Geographic longitude  
- ⚠️ Weak standalone predictive value  

---

#### `loc_latitude`
- Geographic latitude  
- ⚠️ Weak standalone predictive value  

---

---
---

---
---

## Missing values snapshot

In [17]:
missing_summary = (
    df.isna()
    .sum()
    .reset_index()
    .rename(columns={"index": "column", 0: "n_missing"})
    .query("n_missing > 0")
    .sort_values("n_missing", ascending=False)
)

print(f"\nColumns with missing values: {missing_summary.shape[0]}")
missing_summary


Columns with missing values: 4


,column,n_missing
53,st_churn_category,5174
54,st_churn_reason,5174
21,svc_offer,3877
26,svc_internet_type,1526


### Missing Values – Interpretation & Handling

Only four variables exhibit missing values, and none of them are random.  
All missing values are **structurally induced** by business logic or outcome conditions.

In particular, the churn-related variables (st_churn_category and st_churn_reason) are only populated for customers who have churned. As a result, these variables must **not** be imputed and should be excluded from predictive modeling to avoid **target leakage**.

For the service-related variables, missing values reflect the **absence of the corresponding service**, rather than data collection errors. Therefore, missing entries will be replaced with explicit category labels rather than statistical estimates:

- svc_internet_type → filled with "No Internet Service"
- svc_offer → filled with "No Offer" or "Unknown" depending on business interpretation

# Phase 2 - Redundancy Analysis 
Here we **justify** dropping some columns by showing:

- They are constant (or almost constant)  
- Or they are directly derivable from other fields (redundant information)

## Remove redundant columns

In [10]:
#based on EDA conclusions, drop unneeded columns
cols_to_drop = [
    "demo_count",
    "loc_count",
    "svc_count",
    "st_count",
    "loc_country",
    "loc_state",
    "svc_quarter",
    "st_quarter",
    "st_churn_label",
    "st_churn_score",
    "loc_city",
    "loc_lat_long",
    "loc_latitude",
    "loc_longitude",
    "svc_total_charges",
    "svc_total_long_distance_charges",
    "svc_monthly_charge",
]

# Drop only those that exist (defensive)
existing_to_drop = [c for c in cols_to_drop if c in df.columns]

df = df.drop(columns=existing_to_drop)

print("Dropped columns:", existing_to_drop)
print("New shape:", df.shape)


Dropped columns: []
New shape: (7043, 39)


In [11]:
#Handle meaningful missing values in categoricals
# Fill missing values with explicit categories where appropriate
fill_map = {}

if "svc_offer" in df.columns:
    fill_map["svc_offer"] = "No Offer"

if "svc_internet_type" in df.columns:
    fill_map["svc_internet_type"] = "No Internet"

if "st_churn_category" in df.columns:
    fill_map["st_churn_category"] = "Not Churned"

if "st_churn_reason" in df.columns:
    fill_map["st_churn_reason"] = "Not Churned"

df = df.fillna(value=fill_map)

# Quick check
df[["svc_offer", "svc_internet_type", "st_churn_category", "st_churn_reason"]].head()


,svc_offer,svc_internet_type,st_churn_category,st_churn_reason
0,No Offer,DSL,Competitor,Competitor offered more data
1,Offer E,Fiber Optic,Competitor,Competitor made better offer
2,Offer D,Fiber Optic,Competitor,Competitor made better offer
3,Offer C,Fiber Optic,Dissatisfaction,Limited range of services
4,Offer C,Fiber Optic,Price,Extra data charges


In [12]:
#Ensure Yes/No consistency AND convert binary fields to 0/1

binary_cols = [
    "demo_under_30",
    "demo_senior_citizen",
    "demo_married",
    "demo_dependents",
    "svc_referred_a_friend",
    "svc_phone_service",
    "svc_multiple_lines",
    "svc_internet_service",
    "svc_online_security",
    "svc_online_backup",
    "svc_device_protection_plan",
    "svc_premium_tech_support",
    "svc_streaming_tv",
    "svc_streaming_movies",
    "svc_streaming_music",
    "svc_unlimited_data",
    "svc_paperless_billing",
]

# Standardize Yes/No → Yes/No
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()  # e.g., "yes" → "Yes"

# Convert Yes/No → 1/0
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map({"Yes": 1, "No": 0})

# Quick check
print("Binary columns converted to 0/1:")
for col in binary_cols:
    if col in df.columns:
        print(col, df[col].unique())



Binary columns converted to 0/1:
demo_under_30 [nan]
demo_senior_citizen [nan]
demo_married [nan]
demo_dependents [nan]
svc_referred_a_friend [nan]
svc_phone_service [nan]
svc_multiple_lines [nan]
svc_internet_service [nan]
svc_online_security [nan]
svc_online_backup [nan]
svc_device_protection_plan [nan]
svc_premium_tech_support [nan]
svc_streaming_tv [nan]
svc_streaming_movies [nan]
svc_streaming_music [nan]
svc_unlimited_data [nan]
svc_paperless_billing [nan]


In [13]:
# Convert demo_number_of_dependents into a binary feature

if "demo_number_of_dependents" in df.columns:
    df["demo_has_dependents"] = df["demo_number_of_dependents"].apply(lambda x: 1 if x > 0 else 0)

    # Optionally drop the original skewed variable
    df = df.drop(columns=["demo_number_of_dependents"])

# Quick check
print("demo_has_dependents unique values:", df["demo_has_dependents"].unique())


demo_has_dependents unique values: [0 1]


In [14]:
sys.path.append(os.path.abspath(".."))


NameError: name 'os' is not defined

In [ ]:
# === 1. Imports ===
import os
import pandas as pd
import numpy as np
from src.utils_data import load_df, save_df

# === 2. Configurações e caminhos ===
DATA_DIR = os.path.join("..", "data")
RAW_MERGED = os.path.join(DATA_DIR, "interim", "raw_merged.csv")
CLEANED = os.path.join(DATA_DIR, "interim", "cleaned_df.csv")

# === 3. Carregar dados ===
df = pd.read_csv(RAW_MERGED)
print(f"Shape inicial: {df.shape}")
df.head()


In [ ]:
# Normalizar nomes de colunas
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "", regex=False)
)

# Remover colunas irrelevantes ou duplicadas
drop_cols = ["count"]  # já veio repetida nos merges
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# Converter colunas numéricas
num_cols = ["totalcharges", "monthlycharge", "tenureinmonths", "churnscore", "cltv"]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Corrigir nulos
df = df.fillna({
    "offer": "None",
    "internettype": "Unknown",
    "churncategory": "Unknown",
    "churnreason": "Unknown"
})
df = df.fillna(0)


In [ ]:
#Quick sanity check after cleaning
print("Shape after cleaning:", df.shape)
print("\nMissing values (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))


Shape after cleaning: (7043, 39)

Missing values (top 10):
demo_gender                0
demo_age                   0
demo_under_30              0
demo_senior_citizen        0
demo_married               0
demo_dependents            0
loc_zip_code               0
svc_referred_a_friend      0
svc_number_of_referrals    0
svc_tenure_in_months       0
dtype: int64


In [ ]:
#Save cleaned dataset to interim folder
CLEAN_PATH = "../data/interim/telco_master_clean.csv"

df.to_csv(CLEAN_PATH, index=False)

print(f"Cleaned dataset saved to: {CLEAN_PATH}")


Cleaned dataset saved to: ../data/interim/telco_master_clean.csv


In [ ]:
# Data Type Summary + Small-Cardinality Category Check
# Step: Summarize data types
dtype_summary = (
    df.dtypes
    .reset_index()
    .rename(columns={'index': 'column', 0: 'dtype'})
    .sort_values('dtype')
)

print("=== Data Type Summary ===")
display(dtype_summary)


# Step: Identify categorical columns AFTER cleaning
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("\n=== Categorical Columns ===")
print(cat_cols)


# Step: Show unique values for categorical columns with <5 unique entries
print("\n=== Categorical Variables with <5 Unique Values ===")
for col in cat_cols:
    unique_vals = df[col].unique()
    if len(unique_vals) <= 5:
        print(f"\nColumn: {col}  (unique={len(unique_vals)})")
        print(unique_vals)


=== Data Type Summary ===


,column,dtype
19,svc_device_protection_plan,int64
34,st_cltv,int64
33,st_churn_value,int64
31,st_satisfaction_score,int64
29,svc_total_extra_data_charges,int64
26,svc_paperless_billing,int64
24,svc_unlimited_data,int64
23,svc_streaming_music,int64
22,svc_streaming_movies,int64
21,svc_streaming_tv,int64



=== Categorical Columns ===
['demo_gender', 'svc_offer', 'svc_internet_type', 'svc_contract', 'svc_payment_method', 'st_customer_status', 'st_churn_category', 'st_churn_reason']

=== Categorical Variables with <5 Unique Values ===

Column: demo_gender  (unique=2)
['Male' 'Female']

Column: svc_internet_type  (unique=4)
['DSL' 'Fiber Optic' 'Cable' 'No Internet']

Column: svc_contract  (unique=3)
['Month-to-Month' 'One Year' 'Two Year']

Column: svc_payment_method  (unique=3)
['Bank Withdrawal' 'Credit Card' 'Mailed Check']

Column: st_customer_status  (unique=3)
['Churned' 'Stayed' 'Joined']


# 1: Dataset Understanding

# 2: Descriptive Statistics

# 3: Outliers Detection

# 4: Correlations

# 5: Visualization of variables and relations

<!-- MODELO: Secção principal numerada -->
<!-- 
<h2 style="background-color:#1f77b4; color:white; padding:10px; border-radius:6px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Secção com linha colorida -->
<!-- 
<h2 style="color:#ff7f0e; border-bottom: 3px solid #ff7f0e; padding-bottom:4px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Subsecção -->
<!-- 
<h3 style="color:#2ca02c; margin-top:10px;">
    X.Y Nome da Subsecção
</h3>
-->

<!-- MODELO: Caixa de Nota -->
<!-- 
<div style="border-left: 5px solid #1f77b4; padding:10px; background-color:#f5f9ff; margin:15px 0;">
    <b>Nota:</b> Texto da nota.
</div>
-->

<!-- MODELO: Secção principal numerada -->
<!-- 
<h2 style="background-color:#1f77b4; color:white; padding:10px; border-radius:6px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Secção com linha colorida -->
<!-- 
<h2 style="color:#ff7f0e; border-bottom: 3px solid #ff7f0e; padding-bottom:4px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Subsecção -->
<!-- 
<h3 style="color:#2ca02c; margin-top:10px;">
    X.Y Nome da Subsecção
</h3>
-->

<!-- MODELO: Caixa de Nota -->
<!-- 
<div style="border-left: 5px solid #1f77b4; padding:10px; background-color:#f5f9ff; margin:15px 0;">
    <b>Nota:</b> Texto da nota.
</div>
-->